In [ ]:
from utils.getter import *
import torch.utils.data as data
import torch
from tqdm import tqdm
import torch.nn as nn

In [ ]:
transforms = Compose([
    Resize((300,300)),
    ToTensor(),
    Normalize(),
])

In [ ]:
data_path = "datasets/datasets/Garbage Classification"
voc_path = "datasets/datasets/VOC/images"
voc_anno = {
    "train": "datasets/datasets/VOC/annotations/pascal_train2012.json",
    "val": "datasets/datasets/VOC/annotations/pascal_val2012.json"}

#trainset = ImageClassificationDataset(data_path+ "/train", transforms= transforms, shuffle=True)
#valset = ImageClassificationDataset(data_path+ "/val", transforms= transforms, shuffle = True)

trainset = ObjectDetectionDataset(img_dir=voc_path, ann_path = voc_anno['train'],transforms= transforms)
valset = ObjectDetectionDataset(img_dir=voc_path, ann_path = voc_anno['val'],transforms= transforms)
NUM_CLASSES = len(trainset.classes)
print(trainset)
print(valset)

In [ ]:
trainset.visualize_item(figsize=(5,5))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

In [ ]:
from models.retinanet.retina_collator import *
my_collator = trainset.collate_fn#RetinaNetCollator()

In [ ]:
# Dataloader
BATCH_SIZE = 4
trainloader = data.DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn = my_collator, shuffle=True)
valloader = data.DataLoader(valset, batch_size=BATCH_SIZE,collate_fn = my_collator, shuffle=False)

In [ ]:
from models.detector import Detector
"""from models.retinanet.loss import FocalLoss
from models.retinanet.model import RetinaNet
criterion = FocalLoss(num_classes = NUM_CLASSES, device = device)
model = RetinaNet(num_classes = NUM_CLASSES).to(device)"""
from models.ssd.model import SSD300, MultiBoxLoss
model = SSD300(n_classes = NUM_CLASSES).to(device)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
print(model)

In [ ]:
for idx, batch in enumerate(trainloader):
    optimizer.zero_grad()
    imgs = batch['imgs'].to(device)
    boxes = [x.to(device) for x in batch['boxes']]
    labels = [x.to(device) for x in batch['labels']]
    
    
    loc_preds, cls_preds = model(imgs)
    loss = criterion(loc_preds,boxes, cls_preds,labels)
  
    #loss = sum([k for k in losses.values()])
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    
    if idx % 10 == 0:
        print('[{}/{}] | Loss: {} '.format(idx, len(trainloader), loss.item()))
        #print('[{}/{}] | C: {} | B: {} '.format(idx, len(trainloader), losses['C'].item(), losses['B'].item()))

In [ ]:
for idx, batch in enumerate(trainloader):
    with torch.no_grad():
        imgs = batch['imgs'].to(device)
        boxes = [x.to(device) for x in batch['boxes']]
        labels = [x.to(device) for x in batch['labels']]
        loc_preds, cls_preds = model(imgs)
        det_boxes, det_labels, det_scores = model.detect_objects(
            loc_preds,
            cls_preds,
            min_score=0.01,
            max_overlap=0.45,
            top_k=200)
    break

In [ ]:
print(det_scores)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam
metrics = [AccuracyMetric(decimals=3)]

In [ ]:
NUM_CLASSES = len(trainset.classes)
model = ResNet34(NUM_CLASSES,
                 lr = 1e-4,
                 criterion= criterion, 
                 optimizer= optimizer,
                 metrics=  metrics,
                 device = device)
#load_checkpoint(model, "weights/ResNet34-12.pth")

In [ ]:
cp = Checkpoint(save_per_epoch=1)
trainer = Trainer(model,
                 trainloader, 
                 valloader,
                 checkpoint = cp, 
                 evaluate_per_epoch = 1)

trainer.fit(num_epochs=5)

In [ ]:
preds = trainer.inference_batch(valloader)
print(preds)

In [ ]:
idx = 35
print(valset.classes[preds[idx]])
valset.visualize_item(idx, figsize = (8,8))